In [1]:
from __future__ import annotations
import polars as pl
import inspect
import numpy as np
from typing import Tuple

In [ ]:
from dsds.metrics import roc_auc
from dsds.transform import multicat_one_hot_encode

In [ ]:
pred = np.random.random(size=1_000_000) # .reshape(-1, 1) # Predictions
actual = np.random.random(size=1_000_000) # Actuals

In [ ]:
pl.show_versions()

In [ ]:
df = pl.DataFrame({
    "text1":["abc|ggg", "abc|sss", "ccc|abc"],
    "text2":["aaa|bbb", "ccc|aaa", "bbb|ccc"]
})
print(df)

In [ ]:
print(multicat_one_hot_encode(df, cols=["text1", "text2"], delimiter="|"))

In [ ]:
test = pl.DataFrame({
    "a":[1,1,1,1]
    , "b": [None, None , "a", "a"]
})
test

In [ ]:
import dsds.fs as fs
import polars as pl
from sklearn.datasets import make_classification

orig_x, orig_y = make_classification(n_samples = 500_000, n_features = 10, n_informative = 5, n_redundant = 5)
df = pl.from_numpy(orig_x).insert_at_idx(0, pl.Series("target", orig_y))
target = "target"
features = df.columns
features.remove(target)

In [ ]:
fs.permutation_importance(df, target="target", model_str="lgbm", params={"max_depth":4})

In [ ]:
fc, fi = fs.ebfs(df, "target", "lgbm", {"max_depth":4}, n_comb=2)

In [2]:
import dsds.transform as t
df = pl.DataFrame({"date1":["2021-01-01", "2022-02-03", "2023-11-23"]
                   , "date2":["2021-01-01", "2022-02-03", "2023-11-23"]})\
    .with_columns(
        pl.col(c).str.to_date() for c in ["date1", "date2"]
    )
print(df)

shape: (3, 2)
┌────────────┬────────────┐
│ date1      ┆ date2      │
│ ---        ┆ ---        │
│ date       ┆ date       │
╞════════════╪════════════╡
│ 2021-01-01 ┆ 2021-01-01 │
│ 2022-02-03 ┆ 2022-02-03 │
│ 2023-11-23 ┆ 2023-11-23 │
└────────────┴────────────┘


In [3]:
cols = ["date1", "date2"]
print(t.extract_dt_features(df, cols=cols))

shape: (3, 8)
┌────────────┬────────────┬────────────┬───────────┬───────────┬───────────┬───────────┬───────────┐
│ date1      ┆ date2      ┆ date1_year ┆ date2_yea ┆ date1_qua ┆ date2_qua ┆ date1_mon ┆ date2_mon │
│ ---        ┆ ---        ┆ ---        ┆ r         ┆ rter      ┆ rter      ┆ th        ┆ th        │
│ date       ┆ date       ┆ u16        ┆ ---       ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
│            ┆            ┆            ┆ u16       ┆ u8        ┆ u8        ┆ u8        ┆ u8        │
╞════════════╪════════════╪════════════╪═══════════╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 2021-01-01 ┆ 2021-01-01 ┆ 2021       ┆ 2021      ┆ 1         ┆ 1         ┆ 1         ┆ 1         │
│ 2022-02-03 ┆ 2022-02-03 ┆ 2022       ┆ 2022      ┆ 1         ┆ 1         ┆ 2         ┆ 2         │
│ 2023-11-23 ┆ 2023-11-23 ┆ 2023       ┆ 2023      ┆ 4         ┆ 4         ┆ 11        ┆ 11        │
└────────────┴────────────┴────────────┴───────────┴───────────┴───────────┴─